In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

df = pd.read_csv("movie_dataset.csv", engine="python")

df = df[['budget', 'popularity', 'runtime', 'vote_average', 'vote_count', 'revenue']]
df = df.dropna()


df.head()


,budget,popularity,runtime,vote_average,vote_count,revenue
0,237000000,150.437577,162,7.2,11800.0,2787965087
1,300000000,139.082615,169,6.9,4500.0,961000000
2,245000000,107.376788,148,6.3,4466.0,880674609
3,250000000,112.31295,165,7.6,9106.0,1084939099
4,260000000,43.926995,132,6.1,2124.0,284139100


In [2]:
numeric_columns = ['budget', 'popularity', 'runtime', 'vote_average', 'vote_count', 'revenue']

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna()


In [3]:
median_revenue = df['revenue'].median()

df['target'] = np.where(df['revenue'] >= median_revenue, 1, 0)

df = df.drop('revenue', axis=1)

In [4]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42
)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [6]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    batch_size=32
)


Epoch 1/50


c:\Users\Clyde\report_IS2\Movie-Dataset\.venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8186 - loss: 0.4197 - val_accuracy: 0.8752 - val_loss: 0.3072
Epoch 2/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8422 - loss: 0.3505 - val_accuracy: 0.8766 - val_loss: 0.3087
Epoch 3/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8498 - loss: 0.3379 - val_accuracy: 0.8793 - val_loss: 0.2841
Epoch 4/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8449 - loss: 0.3377 - val_accuracy: 0.8710 - val_loss: 0.2851
Epoch 5/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8453 - loss: 0.3330 - val_accuracy: 0.8669 - val_loss: 0.3040
Epoch 6/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8458 - loss: 0.3314 - val_accuracy: 0.8877 - val_loss: 0.2785
Epoch 7/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8515 - loss: 0.3286 - val_accuracy: 0.8863 - val_loss: 0.2819
Epoch 8/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8522 - loss: 0.3263 - val_accuracy: 0.8877 - val_

In [7]:
loss, accuracy = model.evaluate(X_val, y_val)
print("\nValidation Accuracy:", accuracy * 100, "%")

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8835 - loss: 0.2726 

Validation Accuracy: 88.34951519966125 %


In [9]:
model.save_weights("movie_revenue_model_weights.weights.h5")
fresh_model = Sequential()
fresh_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
fresh_model.add(Dense(64, activation='relu'))
fresh_model.add(Dense(32, activation='relu'))
fresh_model.add(Dense(16, activation='relu'))
fresh_model.add(Dense(1, activation='sigmoid'))
fresh_model.load_weights("movie_revenue_model_weights.weights.h5")

print("Weights saved and loaded into a fresh architecture!")


Weights saved and loaded into a fresh architecture!


c:\Users\Clyde\report_IS2\Movie-Dataset\.venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
